# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [8]:
import numpy as np
import os
from scipy.misc import imread, imresize
import imageio

import matplotlib.pyplot as plt
import skimage.transform

import datetime
import os
from time import time


We set the random seed so that the results don't vary drastically.

In [9]:
# !pip install --upgrade pip
# !pip install keras
# ! pip install imageio
# ! pip install scikit-image

In [10]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [11]:
# function to normalise the data
def normalize_data(data):
    return data/127.5-1


# function to initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

def load_batch_images(source_path, folder_list, batch_num, batch_size, t ):
    batch_data,batch_labels = init_batch_data(batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, nb_frames)] 
    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])         
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = imageio.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = skimage.transform.resize(image, (nb_rows,nb_cols),preserve_range=True)
            batch_data[folder,idx,:,:,0] = normalize_data(resized[:,:,0])
            batch_data[folder,idx,:,:,1] = normalize_data(resized[:,:,1])
            batch_data[folder,idx,:,:,2] = normalize_data(resized[:,:,2])

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    
    return batch_data, batch_labels

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [12]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield load_batch_images(source_path, folder_list, batch, batch_size, t) 
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_batch_images(source_path, folder_list, num_batches, batch_size, t)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [13]:
# /storage/Gesture_Recognition/gdown.pl-master
# /storage/demo_gesture  
# /content/drive/My Drive/

train_doc = np.random.permutation(open('/storage/demo_gesture/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/storage/demo_gesture/Project_data/val.csv').readlines())

curr_dt_time = datetime.datetime.now()
train_path = '/storage/demo_gesture/Project_data/train'
val_path = '/storage/demo_gesture/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)


# training sequences = 663
# validation sequences = 100


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, ZeroPadding3D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.layers.recurrent import LSTM
from keras.applications import vgg16, inception_v3, resnet50

In [20]:
class ModelGenerator(object):
    
    @classmethod
    def c3d42(cls, input_shape, nb_classes):
        # Model.
        model = Sequential()
        model.add(Conv3D(8, (3,3,3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(16, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

    @classmethod
    def c3d42nor(cls, input_shape, nb_classes):
        # Model.
        model = Sequential()
        model.add(Conv3D(8, (3,3,3), activation='relu', input_shape=input_shape))
#         model.add(BatchNormalization())

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(16, (3,3,3), activation='relu'))
#         model.add(BatchNormalization())

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
#         model.add(BatchNormalization())

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
#         model.add(BatchNormalization())

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

    
    @classmethod
    def c3d42b(cls, input_shape, nb_classes):
        # Model.
        model = Sequential()
        model.add(Conv3D(8, (3,3,3), activation='relu', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(16, (3,3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(64, (3,3,3), activation='relu'))
        model.add(Dropout(0.25))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Dropout(0.5))
        model.add(Dense(128))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))
        return model
    
    
    @classmethod
    def c3d42bc(cls, input_shape, nb_classes):
        # Model.
        model = Sequential()
        model.add(Conv3D(16, (3,3,3), activation='relu'),input_shape=input_shape)
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Conv3D(32, (1,3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Conv3D(64, (1,3,3), activation='relu'))
        model.add(Dropout(0.25))
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Dropout(0.5))
        model.add(Dense(128))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))
        return model

    @classmethod
    def lrcn(cls, input_shape, nb_classes):
        model = Sequential()
        model.add(TimeDistributed(Conv2D(16, (7, 7), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))
        model.add(TimeDistributed(Conv2D(16, (3,3), kernel_initializer="he_normal", activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
        model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(256, return_sequences=False, dropout=0.5))
        model.add(Dense(nb_classes, activation='softmax'))
        return model
    
    @classmethod
    def lrcn2(cls, input_shape, nb_classes):
        model = Sequential()
        model.add(TimeDistributed(Conv2D(16, (7, 7), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))
        model.add(TimeDistributed(Conv2D(16, (3,3), kernel_initializer="he_normal", activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
        model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(256, return_sequences=False, dropout=0.5))
        model.add(Dense(nb_classes, activation='softmax'))
        return model

In [16]:
def train(batch_size, num_epochs, model, train_generator, val_generator, optimiser=None):

    curr_dt_time = datetime.datetime.now()

    num_train_sequences = len(train_doc)
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    print('# batch size =', batch_size)    
    print('# epochs =', num_epochs)

    #optimizer = Adam(lr=rate) 
    #write your optimizer
    if optimiser == None:
        optimiser = Adam() 
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)
            
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=False, 
                                 save_weights_only=False, 
                                 mode='auto', 
                                 period=1)
    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
    callbacks_list = [checkpoint, LR]

    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1

    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                callbacks=callbacks_list, validation_data=val_generator, 
                validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
    
    K.clear_session()

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
batch_size = 90 #experiment with the batch size
num_epochs = 10 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
print ('# epochs =', num_epochs)

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)
num_classes = 5


input_shape = (nb_frames, nb_rows, nb_cols, 3)
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()



model = model_gen.c3d42(input_shape, num_classes)

train(batch_size, num_epochs, model, train_generator, val_generator)

# epochs = 10
# training sequences = 663
# validation sequences = 100
# batch size = 90
# epochs = 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 18, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 18, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 16, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 16, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 14, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 12, 24, 24, 32)    27680     
________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


8/8 [==============================] - 130s 16s/step - loss: 1.8345 - categorical_accuracy: 0.1986 - val_loss: 1.5664 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_2018-12-3011_55_59.616133/model-00001-1.85241-0.19759-1.56639-0.34000.h5
Epoch 2/10
8/8 [==============================] - 34s 4s/step - loss: 1.5679 - categorical_accuracy: 0.2689 - val_loss: 1.3942 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to model_init_2018-12-3011_55_59.616133/model-00002-1.56787-0.26894-1.39417-0.35000.h5
Epoch 3/10
8/8 [==============================] - 34s 4s/step - loss: 1.5151 - categorical_accuracy: 0.2841 - val_loss: 1.5022 - val_categorical_accuracy: 0.4500

Epoch 00003: saving model to model_init_2018-12-3011_55_59.616133/model-00003-1.51515-0.28409-1.50219-0.45000.h5
Epoch 4/10
8/8 [==============================] - 18s 2s/step - loss: 1.5562 - categorical_accuracy: 0.2282 - val_loss: 1.4619 - val_categorical_accuracy: 0.2500

Epoch 00004: savin

In [17]:
batch_size = 50 #experiment with the batch size
num_epochs = 20 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)
num_classes = 5


input_shape = (nb_frames, nb_rows, nb_cols, 3)
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()



model = model_gen.c3d42(input_shape, num_classes)

train(batch_size, num_epochs, model, train_generator, val_generator)

# epochs = 20
# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 18, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 18, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 16, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 16, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 14, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 12, 24, 24, 32)    27680     
________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 100s 7s/step - loss: 1.8655 - categorical_accuracy: 0.2125 - val_loss: 1.5611 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-12-3012_01_42.995160/model-00001-1.88021-0.20362-1.56109-0.23000.h5
Epoch 2/20
14/14 [==============================] - 25s 2s/step - loss: 1.5781 - categorical_accuracy: 0.2747 - val_loss: 1.5681 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to model_init_2018-12-3012_01_42.995160/model-00002-1.57813-0.27473-1.56805-0.36000.h5
Epoch 3/20
14/14 [==============================] - 26s 2s/step - loss: 1.4641 - categorical_accuracy: 0.4121 - val_loss: 1.5396 - val_categorical_accuracy: 0.3200

Epoch 00003: saving model to model_init_2018-12-3012_01_42.995160/model-00003-1.46406-0.41209-1.53959-0.32000.h5
Epoch 4/20
14/14 [==============================] - 26s 2s/step - loss: 1.5484 - categorical_accuracy: 0.3077 - val_loss: 1.4652 - val_categorical_accuracy: 0.2000

Epoch 00004

In [21]:
%%time
batch_size = 30 #experiment with the batch size
num_epochs = 20 # choose the number of epochs
nb_frames=15
nb_rows=120
nb_cols=120
nb_channel=3

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)
num_classes = 5


input_shape = (nb_frames, nb_rows, nb_cols, 3)
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()



model = model_gen.c3d42(input_shape, num_classes)

train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 30
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 13, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 11, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 11, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 9, 26, 26, 32)     13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 7, 24, 24, 32)     27680     
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


23/23 [==============================] - 67s 3s/step - loss: 1.6303 - categorical_accuracy: 0.2274 - val_loss: 1.5963 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2018-12-3012_14_35.171202/model-00001-1.63205-0.22323-1.59632-0.20000.h5
Epoch 2/20
23/23 [==============================] - 6s 257ms/step - loss: 1.6221 - categorical_accuracy: 0.1014 - val_loss: 1.6237 - val_categorical_accuracy: 0.1750

Epoch 00002: saving model to model_init_2018-12-3012_14_35.171202/model-00002-1.62212-0.10145-1.62371-0.17500.h5
Epoch 3/20
23/23 [==============================] - 7s 293ms/step - loss: 1.6178 - categorical_accuracy: 0.1884 - val_loss: 1.6326 - val_categorical_accuracy: 0.1500

Epoch 00003: saving model to model_init_2018-12-3012_14_35.171202/model-00003-1.61777-0.18841-1.63257-0.15000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/20
23/23 [==============================] - 7s 298ms/step - loss: 1.6331 - categoric

In [23]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 20 # choose the number of epochs
nb_frames=30
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d2(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), padding="same", name="conv1", strides=(1, 1, 1), input_shape=(30, 120, ..., activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(strides=(1, 2, 2), name="pool1", padding="valid", pool_size=(1, 2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), padding="same", name="conv2", strides=(1, 1, 1), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(strides=(2, 2, 2), name="pool2", padding="valid", pool_size=(2, 2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv3D` call to th

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 160s 11s/step - loss: 1.6128 - categorical_accuracy: 0.1892 - val_loss: 1.6056 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-12-3012_24_12.395698/model-00001-1.61295-0.18703-1.60565-0.23000.h5
Epoch 2/20
14/14 [==============================] - 37s 3s/step - loss: 1.5986 - categorical_accuracy: 0.2088 - val_loss: 1.5927 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2018-12-3012_24_12.395698/model-00002-1.59858-0.20879-1.59269-0.18000.h5
Epoch 3/20
14/14 [==============================] - 37s 3s/step - loss: 1.6064 - categorical_accuracy: 0.1758 - val_loss: 1.6095 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2018-12-3012_24_12.395698/model-00003-1.60635-0.17582-1.60949-0.21000.h5
Epoch 4/20
14/14 [==============================] - 38s 3s/step - loss: 1.6104 - categorical_accuracy: 0.1923 - val_loss: 1.6090 - val_categorical_accuracy: 0.2200

Epoch 0000

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [24]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 25 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d42b(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 25
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 20, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 40, 40, 16)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 40, 40, 16)     0         
______________________________________________________

In [33]:
%%time
batch_size = 90 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=30
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d42b(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 18, 118, 118, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 18, 118, 118, 8)   32        
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 18, 59, 59, 8)     0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 16, 57, 57, 16)    3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 16, 57, 57, 16)    64        
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 16, 28, 28, 16)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 114s 8s/step - loss: 11.9414 - categorical_accuracy: 0.1959 - val_loss: 9.3642 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2018-12-3014_39_19.183891/model-00001-11.93297-0.20211-9.36417-0.41000.h5
Epoch 2/20
14/14 [==============================] - 25s 2s/step - loss: 11.5249 - categorical_accuracy: 0.2802 - val_loss: 11.1215 - val_categorical_accuracy: 0.3100

Epoch 00002: saving model to model_init_2018-12-3014_39_19.183891/model-00002-11.52486-0.28022-11.12149-0.31000.h5
Epoch 3/20
14/14 [==============================] - 26s 2s/step - loss: 12.4871 - categorical_accuracy: 0.2253 - val_loss: 11.5395 - val_categorical_accuracy: 0.2800

Epoch 00003: saving model to model_init_2018-12-3014_39_19.183891/model-00003-12.48710-0.22527-11.53952-0.28000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/20
14/14 [==============================] - 26s 2s/step - loss: 12.7528 - cat

In [17]:
%%time
batch_size = 90 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d42b(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 90
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 18, 118, 118, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 18, 118, 118, 8)   32        
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 18, 59, 59, 8)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 16, 57, 57, 16)    3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 16, 57, 57, 16)    64        
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 16, 28, 28, 16)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


8/8 [==============================] - 94s 12s/step - loss: 11.6014 - categorical_accuracy: 0.2253 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2018-12-3015_04_07.931396/model-00001-11.54846-0.22624-12.57212-0.22000.h5
Epoch 2/30
8/8 [==============================] - 32s 4s/step - loss: 11.9517 - categorical_accuracy: 0.2538 - val_loss: 11.6543 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2018-12-3015_04_07.931396/model-00002-11.95170-0.25379-11.65433-0.25000.h5
Epoch 3/30
8/8 [==============================] - 33s 4s/step - loss: 11.3838 - categorical_accuracy: 0.2917 - val_loss: 14.5063 - val_categorical_accuracy: 0.1000

Epoch 00003: saving model to model_init_2018-12-3015_04_07.931396/model-00003-11.38382-0.29167-14.50629-0.10000.h5
Epoch 4/30
8/8 [==============================] - 19s 2s/step - loss: 13.1649 - categorical_accuracy: 0.1832 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch

Epoch 27/30
8/8 [==============================] - 3s 407ms/step - loss: 14.7749 - categorical_accuracy: 0.0833 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00027: saving model to model_init_2018-12-3015_04_07.931396/model-00027-14.77492-0.08333-13.70038-0.15000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
Epoch 28/30
8/8 [==============================] - 4s 448ms/step - loss: 14.1033 - categorical_accuracy: 0.1250 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00028: saving model to model_init_2018-12-3015_04_07.931396/model-00028-14.10333-0.12500-12.89448-0.20000.h5
Epoch 29/30
8/8 [==============================] - 4s 453ms/step - loss: 10.7454 - categorical_accuracy: 0.3333 - val_loss: 11.2827 - val_categorical_accuracy: 0.3000

Epoch 00029: saving model to model_init_2018-12-3015_04_07.931396/model-00029-10.74540-0.33333-11.28267-0.30000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.220703182

In [28]:
%%time
batch_size = 90 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.lrcn(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 90
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 20, 60, 60, 16)    2368      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 58, 58, 16)    2320      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 29, 29, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 29, 29, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 29, 29, 32)    9248      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 14, 14, 32)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


8/8 [==============================] - 99s 12s/step - loss: 1.6011 - categorical_accuracy: 0.1959 - val_loss: 1.5606 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2018-12-3015_31_24.864941/model-00001-1.60341-0.19005-1.56061-0.41000.h5
Epoch 2/30
8/8 [==============================] - 29s 4s/step - loss: 1.5484 - categorical_accuracy: 0.3182 - val_loss: 1.5166 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2018-12-3015_31_24.864941/model-00002-1.54845-0.31818-1.51658-0.30000.h5
Epoch 3/30
8/8 [==============================] - 34s 4s/step - loss: 1.4967 - categorical_accuracy: 0.3258 - val_loss: 1.4407 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2018-12-3015_31_24.864941/model-00003-1.49665-0.32576-1.44073-0.40000.h5
Epoch 4/30
8/8 [==============================] - 17s 2s/step - loss: 1.3604 - categorical_accuracy: 0.3587 - val_loss: 1.7474 - val_categorical_accuracy: 0.3500

Epoch 00004: saving

In [29]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.lrcn(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 20, 60, 60, 16)    2368      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 58, 58, 16)    2320      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 29, 29, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 29, 29, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 29, 29, 32)    9248      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 14, 14, 32)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 105s 8s/step - loss: 1.6043 - categorical_accuracy: 0.2121 - val_loss: 1.5433 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-12-3015_37_00.755637/model-00001-1.60715-0.21116-1.54326-0.23000.h5
Epoch 2/30
14/14 [==============================] - 25s 2s/step - loss: 1.5758 - categorical_accuracy: 0.2033 - val_loss: 1.6095 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2018-12-3015_37_00.755637/model-00002-1.57579-0.20330-1.60950-0.21000.h5
Epoch 3/30
14/14 [==============================] - 27s 2s/step - loss: 1.6278 - categorical_accuracy: 0.2308 - val_loss: 1.6064 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2018-12-3015_37_00.755637/model-00003-1.62782-0.23077-1.60644-0.21000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/30
14/14 [==============================] - 26s 2s/step - loss: 1.6086 - categorical_ac

In [34]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.lrcn2(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 20, 60, 60, 16)    2368      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 58, 58, 16)    2320      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 29, 29, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 29, 29, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 29, 29, 32)    9248      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 14, 14, 32)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 101s 7s/step - loss: 1.6114 - categorical_accuracy: 0.1854 - val_loss: 1.5982 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-12-3016_00_15.099654/model-00001-1.61135-0.18703-1.59825-0.23000.h5
Epoch 2/30
14/14 [==============================] - 25s 2s/step - loss: 1.5860 - categorical_accuracy: 0.2912 - val_loss: 1.5826 - val_categorical_accuracy: 0.1700

Epoch 00002: saving model to model_init_2018-12-3016_00_15.099654/model-00002-1.58598-0.29121-1.58255-0.17000.h5
Epoch 3/30
14/14 [==============================] - 24s 2s/step - loss: 1.6259 - categorical_accuracy: 0.1703 - val_loss: 1.6142 - val_categorical_accuracy: 0.1600

Epoch 00003: saving model to model_init_2018-12-3016_00_15.099654/model-00003-1.62594-0.17033-1.61425-0.16000.h5
Epoch 4/30
14/14 [==============================] - 26s 2s/step - loss: 1.6133 - categorical_accuracy: 0.1758 - val_loss: 1.6138 - val_categorical_accuracy: 0.2100

Epoch 00004

CPU times: user 20min 33s, sys: 33min 1s, total: 53min 34s
Wall time: 15min 44s


In [17]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d42nor(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 18, 118, 118, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 18, 118, 118, 8)   32        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 18, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 16, 57, 57, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 57, 57, 16)    64        
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 16, 28, 28, 16)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 116s 8s/step - loss: 10.6136 - categorical_accuracy: 0.2948 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-12-3016_41_50.683817/model-00001-10.58434-0.29412-12.41093-0.23000.h5
Epoch 2/30
14/14 [==============================] - 25s 2s/step - loss: 10.9950 - categorical_accuracy: 0.3132 - val_loss: 11.9274 - val_categorical_accuracy: 0.2600

Epoch 00002: saving model to model_init_2018-12-3016_41_50.683817/model-00002-10.99496-0.31319-11.92739-0.26000.h5
Epoch 3/30
14/14 [==============================] - 25s 2s/step - loss: 11.0701 - categorical_accuracy: 0.3132 - val_loss: 11.2827 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2018-12-3016_41_50.683817/model-00003-11.07012-0.31319-11.28267-0.30000.h5
Epoch 4/30
14/14 [==============================] - 26s 2s/step - loss: 10.1819 - categorical_accuracy: 0.3626 - val_loss: 9.8704 - val_categorical_accuracy: 0.3800

14/14 [==============================] - 25s 2s/step - loss: 11.1429 - categorical_accuracy: 0.3077 - val_loss: 10.4768 - val_categorical_accuracy: 0.3500

Epoch 00027: saving model to model_init_2018-12-3016_41_50.683817/model-00027-11.14285-0.30769-10.47676-0.35000.h5
Epoch 28/30
14/14 [==============================] - 23s 2s/step - loss: 11.0420 - categorical_accuracy: 0.3132 - val_loss: 10.4768 - val_categorical_accuracy: 0.3500

Epoch 00028: saving model to model_init_2018-12-3016_41_50.683817/model-00028-11.04201-0.31319-10.47676-0.35000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
Epoch 29/30
14/14 [==============================] - 25s 2s/step - loss: 10.1637 - categorical_accuracy: 0.3681 - val_loss: 10.4768 - val_categorical_accuracy: 0.3500

Epoch 00029: saving model to model_init_2018-12-3016_41_50.683817/model-00029-10.16370-0.36813-10.47676-0.35000.h5
Epoch 30/30
14/14 [==============================] - 25s 2s/step - loss: 10.3616 

In [21]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=20
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d42nor(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 18, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 18, 59, 59, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 16, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 16, 28, 28, 16)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 14, 26, 26, 32)    13856     
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 14, 13, 13, 32)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 98s 7s/step - loss: 3.8838 - categorical_accuracy: 0.2050 - val_loss: 1.6083 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2018-12-3017_01_34.044611/model-00001-4.00154-0.21569-1.60832-0.28000.h5
Epoch 2/30
14/14 [==============================] - 23s 2s/step - loss: 1.5928 - categorical_accuracy: 0.2527 - val_loss: 1.5716 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2018-12-3017_01_34.044611/model-00002-1.59276-0.25275-1.57158-0.24000.h5
Epoch 3/30
14/14 [==============================] - 24s 2s/step - loss: 1.5981 - categorical_accuracy: 0.2363 - val_loss: 1.5252 - val_categorical_accuracy: 0.2400

Epoch 00003: saving model to model_init_2018-12-3017_01_34.044611/model-00003-1.59808-0.23626-1.52518-0.24000.h5
Epoch 4/30
14/14 [==============================] - 24s 2s/step - loss: 1.5254 - categorical_accuracy: 0.2692 - val_loss: 1.5567 - val_categorical_accuracy: 0.3200

Epoch 00004:

Epoch 29/30
14/14 [==============================] - 25s 2s/step - loss: 0.4812 - categorical_accuracy: 0.8077 - val_loss: 0.9615 - val_categorical_accuracy: 0.6700

Epoch 00029: saving model to model_init_2018-12-3017_01_34.044611/model-00029-0.48116-0.80769-0.96155-0.67000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 30/30
14/14 [==============================] - 24s 2s/step - loss: 0.4115 - categorical_accuracy: 0.8516 - val_loss: 0.9636 - val_categorical_accuracy: 0.6600

Epoch 00030: saving model to model_init_2018-12-3017_01_34.044611/model-00030-0.41147-0.85165-0.96360-0.66000.h5
CPU times: user 19min 12s, sys: 27min 21s, total: 46min 34s
Wall time: 13min 58s


In [26]:
%%time
batch_size = 50 #experiment with the batch size
num_epochs = 30 # choose the number of epochs
nb_frames=30
nb_rows=120
nb_cols=120
nb_channel=3
num_classes = 5

input_shape = (nb_frames,nb_rows,nb_cols,nb_channel)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_gen = ModelGenerator()

model = model_gen.c3d42nor(input_shape, num_classes)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 30
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 24, 13, 13, 32)    0         
______________________________________________________

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


14/14 [==============================] - 157s 11s/step - loss: 3.0280 - categorical_accuracy: 0.2585 - val_loss: 1.5576 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-12-3017_49_21.258580/model-00001-3.10194-0.26395-1.55764-0.23000.h5
Epoch 2/30
14/14 [==============================] - 36s 3s/step - loss: 1.5426 - categorical_accuracy: 0.2857 - val_loss: 1.5725 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2018-12-3017_49_21.258580/model-00002-1.54264-0.28571-1.57253-0.24000.h5
Epoch 3/30
14/14 [==============================] - 39s 3s/step - loss: 1.5430 - categorical_accuracy: 0.2857 - val_loss: 1.5001 - val_categorical_accuracy: 0.3600

Epoch 00003: saving model to model_init_2018-12-3017_49_21.258580/model-00003-1.54301-0.28571-1.50012-0.36000.h5
Epoch 4/30
14/14 [==============================] - 38s 3s/step - loss: 1.5283 - categorical_accuracy: 0.3297 - val_loss: 1.4023 - val_categorical_accuracy: 0.4600

Epoch 0000


Epoch 00028: saving model to model_init_2018-12-3017_49_21.258580/model-00028-0.28180-0.88462-1.08321-0.62000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 29/30
14/14 [==============================] - 38s 3s/step - loss: 0.1791 - categorical_accuracy: 0.9286 - val_loss: 1.0832 - val_categorical_accuracy: 0.6300

Epoch 00029: saving model to model_init_2018-12-3017_49_21.258580/model-00029-0.17910-0.92857-1.08318-0.63000.h5
Epoch 30/30
14/14 [==============================] - 39s 3s/step - loss: 0.2533 - categorical_accuracy: 0.9066 - val_loss: 1.0837 - val_categorical_accuracy: 0.6400

Epoch 00030: saving model to model_init_2018-12-3017_49_21.258580/model-00030-0.25334-0.90659-1.08368-0.64000.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
CPU times: user 28min 39s, sys: 43min 56s, total: 1h 12min 35s
Wall time: 21min 46s
